## 条件随机场

区别于**隐马尔可夫**这样的**概率有向图**和**生成式模型**，**条件随机场**是一种**概率无向图**和**判别式模型**。

条件随机场是在给定一组输入随机变量的条件下，另一组输出随机变量的条件概率模型，并且该组输出随机变量构成马尔可夫随机场。

条件随机场的三大问题和相应解法：
- 概率计算问题与前向/后向算法
- 参数估计问题与迭代尺度算法
- 序列标注问题与维特比算法

In [12]:
import nltk
nltk.download('conll2002') # 基于NLTK下载示例数据集

[nltk_data] Error loading conll2002: [WinError 10060]
[nltk_data]     由于连接方在一段时间后没有正确答复或连接的主机没有反应，连接尝试失败。


False

In [13]:
# 设置训练和测试样本
train_sents = list(nltk.corpus.conll2002.iob_sents('esp.train'))
test_sents = list(nltk.corpus.conll2002.iob_sents('esp.testb'))
train_sents[0]

[('Melbourne', 'NP', 'B-LOC'),
 ('(', 'Fpa', 'O'),
 ('Australia', 'NP', 'B-LOC'),
 (')', 'Fpt', 'O'),
 (',', 'Fc', 'O'),
 ('25', 'Z', 'O'),
 ('may', 'NC', 'O'),
 ('(', 'Fpa', 'O'),
 ('EFE', 'NC', 'B-ORG'),
 (')', 'Fpt', 'O'),
 ('.', 'Fp', 'O')]

In [14]:
# 单词转换为数值特征
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }

    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

sent2features(train_sents[0])

[{'bias': 1.0,
  'word.lower()': 'melbourne',
  'word[-3:]': 'rne',
  'word[-2:]': 'ne',
  'word.isupper()': False,
  'word.istitle()': True,
  'word.isdigit()': False,
  'postag': 'NP',
  'postag[:2]': 'NP',
  'BOS': True,
  '+1:word.lower()': '(',
  '+1:word.istitle()': False,
  '+1:word.isupper()': False,
  '+1:postag': 'Fpa',
  '+1:postag[:2]': 'Fp'},
 {'bias': 1.0,
  'word.lower()': '(',
  'word[-3:]': '(',
  'word[-2:]': '(',
  'word.isupper()': False,
  'word.istitle()': False,
  'word.isdigit()': False,
  'postag': 'Fpa',
  'postag[:2]': 'Fp',
  '-1:word.lower()': 'melbourne',
  '-1:word.istitle()': True,
  '-1:word.isupper()': False,
  '-1:postag': 'NP',
  '-1:postag[:2]': 'NP',
  '+1:word.lower()': 'australia',
  '+1:word.istitle()': True,
  '+1:word.isupper()': False,
  '+1:postag': 'NP',
  '+1:postag[:2]': 'NP'},
 {'bias': 1.0,
  'word.lower()': 'australia',
  'word[-3:]': 'lia',
  'word[-2:]': 'ia',
  'word.isupper()': False,
  'word.istitle()': True,
  'word.isdigit()': F

In [20]:
# 构造训练集和测试集
X_train = [sent2features(s) for s in train_sents]
y_train = [sent2labels(s) for s in train_sents]

X_test = [sent2features(s) for s in test_sents]
y_test = [sent2labels(s) for s in test_sents]

len(X_train), len(X_test)

(8323, 1517)

In [26]:
import sklearn_crfsuite
from sklearn import metrics

# 创建CRF模型实例
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)

crf.fit(X_train, y_train) # 模型训练
labels = list(crf.classes_) # 类别标签
y_pred = crf.predict(X_test) # 模型预测

In [25]:
[X['word.lower()'] for X in X_test[0]], y_test[0], y_pred[0]

(['la', 'coruña', ',', '23', 'may', '(', 'efecom', ')', '.'],
 ['B-LOC', 'I-LOC', 'O', 'O', 'O', 'O', 'B-ORG', 'O', 'O'],
 ['B-LOC', 'I-LOC', 'O', 'O', 'O', 'O', 'B-ORG', 'O', 'O'])